In [1]:
from parse_documents import *

# filepath = 'upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg'  # Replace with the path of your file
# filepath = "upload/World_Energy_By_Country_And_Region_1965_to_2023.csv"
filepath = 'upload/Diabetic-retinopathy-identification-using-parallel-convo_2023_Expert-Systems.pdf'
parse_document(filepath)



2024-09-11 00:49:37,396 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2024-09-11 00:50:36,566 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
2024-09-11 00:50:38,172 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


<class 'dict'>
dict_keys(['metadata', 'content', 'status'])
{'pdf:PDFVersion': '1.7', 'xmp:CreatorTool': 'Elsevier', 'pdf:docinfo:title': 'Diabetic retinopathy identification using parallel convolutional neural network based feature extractor and ELM classifier', 'pdf:hasXFA': 'false', 'access_permission:can_print_degraded': 'true', 'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'pdf:num3DAnnotations': '0', 'dc:format': 'application/pdf; version=1.7', 'pdf:docinfo:custom:robots': 'noindex', 'pdf:docinfo:creator_tool': 'Elsevier', 'access_permission:fill_in_form': 'true', 'pdf:docinfo:custom:CreationDate--Text': '7th February 2023', 'pdf:hasCollection': 'false', 'pdf:encrypted': 'false', 'dc:title': 'Diabetic retinopathy identification using parallel convolutional neural network based feature extractor and ELM classifier', 'pdf:containsNonEmbeddedFont': 'false', 'xmp:CreateD

In [2]:
from tika import parser

def parse_document(filepath):
    try:
        parsed_document = parser.from_file(filepath)
        
        # Check if parsing was successful
        if 'status' in parsed_document and parsed_document['status'] == 200:
            print(f"Parsing successful: {parsed_document['status']}")
        else:
            print(f"Parsing failed with status: {parsed_document.get('status')}")
            return
        
        # Print the type and keys of the parsed document
        print(type(parsed_document))
        print(parsed_document.keys())

        # Extract and print metadata if available
        metadata = parsed_document.get('metadata', {})
        if metadata:
            print("Metadata:")
            print(metadata)
        else:
            print("No metadata found.")
        
        # Print content if needed (commented to avoid large outputs)
        # print(parsed_document.get('content', 'No content available'))
        
    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
parse_document("upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg")


Parsing successful: 200
<class 'dict'>
dict_keys(['metadata', 'content', 'status'])
Metadata:
{'Number of Tables': '4 Huffman tables', 'File Modified Date': 'Wed Sep 11 00:51:00 +00:00 2024', 'Compression Type': 'Baseline', 'Data Precision': '8 bits', 'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.image.JpegParser'], 'Number of Components': '3', 'tiff:ImageLength': '623', 'resourceName': "b'Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_n\\xc3\\xa9e_de_Parseval.jpg'", 'Component 2': 'Cb component: Quantization table 1, Sampling factors 1 horiz/1 vert', 'Component 1': 'Y component: Quantization table 0, Sampling factors 2 horiz/2 vert', 'Image Height': '623 pixels', 'Image Width': '500 pixels', 'File Size': '72495 bytes', 'Component 3': 'Cr component: Quantization table 1, Sampling factors 1 horiz/1 vert', 'X-TIKA:Parsed-By': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.image.JpegParser'], 'X-TIKA:parse_ti

In [5]:
from tika import parser

def parse_document(filepath):
    try:
        parsed_document = parser.from_file(filepath)
        
        # Check if parsing was successful
        if 'status' in parsed_document and parsed_document['status'] == 200:
            print(f"Parsing successful: {parsed_document['status']}")
        else:
            print(f"Parsing failed with status: {parsed_document.get('status')}")
            return
        
        # Extract and print list of keys in the parsed document
        document_keys = list(parsed_document.keys())
        print("Keys in the parsed document:")
        print(document_keys)

        # Extract metadata keys if available
        metadata = parsed_document.get('metadata', {})
        if metadata:
            metadata_keys = list(metadata.keys())
            print("Keys in metadata:")
            print(metadata_keys)
        else:
            print("No metadata found.")

    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
parse_document("upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg")


Parsing successful: 200
Keys in the parsed document:
['metadata', 'content', 'status']
Keys in metadata:
['Number of Tables', 'File Modified Date', 'Compression Type', 'Data Precision', 'X-TIKA:Parsed-By-Full-Set', 'Number of Components', 'tiff:ImageLength', 'resourceName', 'Component 2', 'Component 1', 'Image Height', 'Image Width', 'File Size', 'Component 3', 'X-TIKA:Parsed-By', 'X-TIKA:parse_time_millis', 'X-TIKA:embedded_depth', 'File Name', 'Content-Length', 'tiff:BitsPerSample', 'tiff:ImageWidth', 'Content-Type']


In [1]:
import os
import json
from tika import parser
from ipfs_functions import *

def parse_documents_in_directory(directory_path):
    temp_dir = "tmp" # Get the system's temporary directory
    print(f"Saving metadata files in temporary directory: {temp_dir}")

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        
        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            try:
                parsed_document = parser.from_file(file_path)
                
                # Check if parsing was successful
                if 'status' in parsed_document and parsed_document['status'] == 200:
                    metadata = parsed_document.get('metadata', {})
                    upload_json_to_ipfs(metadata)
                    
                    # Save metadata as JSON
                    if metadata:
                        json_filename = f"{os.path.splitext(filename)[0]}.json"
                        json_path = os.path.join(temp_dir, json_filename)

                        with open(json_path, 'w') as json_file:
                            json.dump(metadata, json_file, indent=4)
                        
                        print(f"Metadata saved for '{filename}' as '{json_filename}'")
                    else:
                        print(f"No metadata found for '{filename}'")
                else:
                    print(f"Parsing failed for '{filename}' with status: {parsed_document.get('status')}")
            
            except Exception as e:
                print(f"An error occurred with file '{filename}': {e}")

# Example usage
parse_documents_in_directory("upload")


ConnectionError: ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5001): Max retries exceeded with url: /api/v0/version?stream-channels=true (Caused by NewConnectionError('<ipfshttpclient.requests_wrapper.HTTPConnection object at 0x7f075c5afac0>: Failed to establish a new connection: [Errno 111] Connection refused'))